In [2]:
import pandas as pd 
import numpy as np 
import os 

In [128]:
data_path = '../RNAseq_out/Blobel-SA-14025_2023_04_18/star_rsem/'
Blobel_files = os.listdir(data_path)
Blobel_files = [ i for i in Blobel_files if 'genes.result' in i ]

In [137]:
# Read in individual file's expected counts 
Blobel_df = {}
for file in Blobel_files:
    file_name = file.split('.')[0]
    Blobel_df[file_name] = pd.read_table(data_path + file )

# Generate a list of genes 
gene_id = []
for i in list(Blobel_df[file_name]['gene_id']):
    gene_id.append(i.title())

counts_data = pd.DataFrame(index=gene_id)
for file in Blobel_df: 
    counts_data[file] = list(Blobel_df[file]['expected_count'])

counts_data.index = counts_data.index.str.upper()
# counts_data.to_csv('./preprocessing/Blobel_counts_CompiledExpectedCounts_allGene.csv')

In [16]:
counts_data = pd.read_csv('./preprocessing/Blobel_counts_CompiledExpectedCounts_allGene.csv', index_col = 0)
biomart = pd.read_csv("./preprocessing/biomart_eID_symbol.csv", index_col = 0)
mm_name = pd.read_csv("./preprocessing/mm_symbol_name.csv", index_col = 0)

In [81]:
eID = counts_data.index.values.tolist()
eID = [i.split('.', 1)[0] for i in eID]
eID_compile = pd.DataFrame(eID, columns = ['eID'])

eID_sym_name = pd.DataFrame()

for i in range(eID_compile.shape[0]):
    eID = eID_compile.iloc[i,:].values[0]
    if len(biomart[biomart['ensembl_gene_id'] == eID]['mgi_symbol'].values) > 0:
        name = biomart[biomart['ensembl_gene_id'] == eID]['mgi_symbol'].values[0]
    else:
        name = 'no_name'
    eID_sym_name = pd.concat([eID_sym_name, pd.DataFrame([eID, name]).transpose()], axis = 0)
    
eID_sym_name.columns = ["ensembl_gene_id","symbol"]
eID_sym_name = pd.merge(eID_sym_name, mm_name, how='left')
eID_sym_name.to_csv('./preprocessing/eID_sym_name.csv')